In [ ]:
#Assignment 1

#Tasks:
#1. Load the CSV data:

# Move the file from Workspace to DBFS
dbutils.fs.cp("file:/Workspace/Shared/employee_data.csv","dbfs:/FileStore/employee_data.csv")

# Load the file from DBFS
df = spark.read.format("csv").option("header", "true").load("/FileStore/employee_data.csv")
df.show()

# Show the first 10 rows
print("Top 10 rows")
df.show(10)

# Inspect the schema
df.printSchema()

#2. Data Cleaning:

# Remove rows where Salary is less than 55000
df_cleaned = df.filter(df['Salary'] > 55000)

# Filter employees who joined after the year 2020
df_cleaned2 = df_cleaned.filter(df_cleaned['JoiningDate'] > '2020-12-31')

# Show the cleaned data
print("Cleaned Data:")
df_cleaned.show()
df_cleaned2.show()

#3. Data Aggregation:

# Find the average salary by Department
avg_salary = df_cleaned.groupBy('Department').agg({'Salary': 'avg'})
print("Avg salary")
avg_salary.show()

# Count the number of employees in each Department
employee_count = df_cleaned.groupBy('Department').count()
print("Department count")
employee_count.show()

#4. Write the Data to CSV:
df.write.format("csv").option("header","true").save("/workspace/Shared/employeecsv_output")
print("Data written to CSV")


+----------+-------------+----------+-----------+------+
|EmployeeID|         Name|Department|JoiningDate|Salary|
+----------+-------------+----------+-----------+------+
|      1001|     John Doe|        HR| 2021-01-15| 55000|
|      1002|   Jane Smith|        IT| 2020-03-10| 62000|
|      1003|Emily Johnson|   Finance| 2019-07-01| 70000|
|      1004|Michael Brown|        HR| 2018-12-22| 54000|
|      1005| David Wilson|        IT| 2021-06-25| 58000|
|      1006|  Linda Davis|   Finance| 2020-11-15| 67000|
|      1007| James Miller|        IT| 2019-08-14| 65000|
|      1008|Barbara Moore|        HR| 2021-03-29| 53000|
+----------+-------------+----------+-----------+------+

Top 10 rows
+----------+-------------+----------+-----------+------+
|EmployeeID|         Name|Department|JoiningDate|Salary|
+----------+-------------+----------+-----------+------+
|      1001|     John Doe|        HR| 2021-01-15| 55000|
|      1002|   Jane Smith|        IT| 2020-03-10| 62000|
|      1003|Emily 

In [ ]:
#Assignment 2

#Tasks:
#1. Load the JSON data:

dbutils.fs.cp("file:/Workspace/Shared/product_data.json", "dbfs:/FileStore/product_data.json")

# Load 
df = spark.read.option("multiline", "true").json("/FileStore/product_data.json")
df.show()

# Show the first 10 rows
print("Top 10 rows")
df.show(10)

# Inspect the schema
df.printSchema()

#2. Data Cleaning:

#Remove rows where Stock is less than 30.
df_cleaned = df.filter(df['Stock'] >= 30)  

#Filter the products that belong to the "Electronics" category.
df_cleaned = df.filter(df['Category'] == 'Electronics')

#show
print("Cleaned Data:")
df_cleaned.show()

#3. Data Aggregation:

#Calculate the total stock for products in the "Furniture" category.
total_stock = df.filter(df['Category'] == 'Furniture').groupBy('Category').agg({'Stock': 'sum'}).withColumnRenamed('sum(Stock)', 'TotalStock')
print("Total Stock")
total_stock.show()

#Find the average price of all products in the dataset.
avg_price = df.groupBy('Category').agg({'Price': 'avg'}).withColumnRenamed('avg(Price)', 'AvgPrice')
print("Average Price")
avg_price.show()

#4. Write the Data to JSON:
df_cleaned.coalesce(1).write.json('/FileStore/product_data_cleaned.json')
print("Data written to JSON")


+-----------+-----+---------+-----------+-----+
|   Category|Price|ProductID|ProductName|Stock|
+-----------+-----+---------+-----------+-----+
|Electronics| 1200|      101|     Laptop|   35|
|Electronics|  800|      102| Smartphone|   80|
|  Furniture|  150|      103| Desk Chair|   60|
|Electronics|  300|      104|    Monitor|   45|
|  Furniture|  350|      105|       Desk|   25|
+-----------+-----+---------+-----------+-----+

Top 10 rows
+-----------+-----+---------+-----------+-----+
|   Category|Price|ProductID|ProductName|Stock|
+-----------+-----+---------+-----------+-----+
|Electronics| 1200|      101|     Laptop|   35|
|Electronics|  800|      102| Smartphone|   80|
|  Furniture|  150|      103| Desk Chair|   60|
|Electronics|  300|      104|    Monitor|   45|
|  Furniture|  350|      105|       Desk|   25|
+-----------+-----+---------+-----------+-----+

root
 |-- Category: string (nullable = true)
 |-- Price: long (nullable = true)
 |-- ProductID: long (nullable = true)
 |-

In [ ]:
# Assignment 3:

# Tasks:

# 1. Convert CSV and JSON Data to Delta Format:

# Load employee.csv file data
df_employee = spark.read.csv('/FileStore/employee_data.csv', header=True, inferSchema=True).cache()
df_employee.show()

# Load product_data.json file
df = spark.read.option("multiline", "true").json("/FileStore/product_data.json")
df.show()

#save data
df_employee.write.format("delta").mode("overwrite").save("/dbfs/FileStore/delta/employee_data")
df.write.format("delta").mode("overwrite").save("/dbfs/FileStore/delta/product_data")
print("Data saved")

#2. Register Delta Tables:

print("Registered")
spark.sql("CREATE TABLE IF NOT EXISTS employee_delta USING DELTA LOCATION '/dbfs/FileStore/delta/employee_data'")
spark.sql("CREATE TABLE IF NOT EXISTS product_delta USING DELTA LOCATION '/dbfs/FileStore/delta/product_data'")

#3. Data Modifications with Delta Tables:

# Increase salary by 5% for IT department employees
spark.sql("UPDATE employee_delta SET Salary = Salary * 1.05 WHERE Department = 'IT'")

# Delete products where stock is less than 40
spark.sql("DELETE FROM product_delta WHERE Stock < 40")
print("Data modified")

#4. Time Travel with Delta Tables:

# Query the product Delta table to show its state before the delete
# operation (use time travel).
print("Data retrieved")
df_product_version_before_delete = spark.sql("SELECT * FROM product_delta VERSION AS OF 0")
df_product_version_before_delete.show()

# Retrieve the version of the employee Delta table before the salary update.
print("Data retrieved")
df_employee_version_before_update = spark.sql("SELECT * FROM employee_delta VERSION AS OF 0")
df_employee_version_before_update.show()

#5. Query Delta Tables:

# Query the employee Delta table to find the employees in the Finance department.
print("employees in Finance department")
df_finance_employees = spark.sql("SELECT * FROM employee_delta WHERE Department = 'Finance'")
df_finance_employees.show()

# Query the product Delta table to find all products in the Electronics category with a price greater than 500.
print("expensive electronics")
df_expensive_electronics = spark.sql("SELECT * FROM product_delta WHERE Category = 'Electronics' AND Price > 500")
df_expensive_electronics.show()






+----------+-------------+----------+-----------+------+
|EmployeeID|         Name|Department|JoiningDate|Salary|
+----------+-------------+----------+-----------+------+
|      1001|     John Doe|        HR| 2021-01-15| 55000|
|      1002|   Jane Smith|        IT| 2020-03-10| 62000|
|      1003|Emily Johnson|   Finance| 2019-07-01| 70000|
|      1004|Michael Brown|        HR| 2018-12-22| 54000|
|      1005| David Wilson|        IT| 2021-06-25| 58000|
|      1006|  Linda Davis|   Finance| 2020-11-15| 67000|
|      1007| James Miller|        IT| 2019-08-14| 65000|
|      1008|Barbara Moore|        HR| 2021-03-29| 53000|
+----------+-------------+----------+-----------+------+

+-----------+-----+---------+-----------+-----+
|   Category|Price|ProductID|ProductName|Stock|
+-----------+-----+---------+-----------+-----+
|Electronics| 1200|      101|     Laptop|   35|
|Electronics|  800|      102| Smartphone|   80|
|  Furniture|  150|      103| Desk Chair|   60|
|Electronics|  300|      10

In [ ]:
#Databricks Delta Lake Exercise
#Tasks
#1. Create Delta Tables Using 3 Methods

#1. Load the sales_data.csv file into a DataFrame.
# Move the file from Workspace to DBFS
dbutils.fs.cp("file:/Workspace/Shared/Sales_data.csv","dbfs:/FileStore/Sales_data.csv")


True

In [ ]:
# Load the CSV file from DBFS
sales_df = spark.read.format("csv").option("header", "true").load("/FileStore/Sales_data.csv")
sales_df.show()

+-------+----------+----------+--------+--------+-----+
|OrderID| OrderDate|CustomerID| Product|Quantity|Price|
+-------+----------+----------+--------+--------+-----+
|   1001|2024-01-15|      C001|Widget A|      10|25.50|
|   1002|2024-01-16|      C002|Widget B|       5|15.75|
|   1003|2024-01-16|      C001|Widget C|       8|22.50|
|   1004|2024-01-17|      C003|Widget A|      15|25.50|
|   1005|2024-01-18|      C004|Widget D|       7|30.00|
|   1006|2024-01-19|      C002|Widget B|       9|15.75|
|   1007|2024-01-20|      C005|Widget C|      12|22.50|
|   1008|2024-01-21|      C003|Widget A|      10|25.50|
+-------+----------+----------+--------+--------+-----+



In [ ]:
#2. Write the DataFrame as a Delta Table.
sales_df.write.format("delta").mode("overwrite").save("/FileStore/delta/sales_delta")

In [ ]:
#3. Load the customer_data.json file into a DataFrame.
# Move the file from Workspace to DBFS
dbutils.fs.cp("file:/Workspace/Shared/customer_data.json", "dbfs:/FileStore/customer_data.json")



True

In [ ]:
# Load the JSON file from DBFS
customer_df = spark.read.option("multiline", "true").json("/FileStore/customer_data.json")
customer_df.show()

+----------+-------------+------+----------+
|CustomerID| CustomerName|Region|SignupDate|
+----------+-------------+------+----------+
|      C001|     John Doe| North|2022-07-01|
|      C002|   Jane Smith| South|2023-02-15|
|      C003|Emily Johnson|  East|2021-11-20|
|      C004|Michael Brown|  West|2022-12-05|
|      C005|  Linda Davis| North|2023-03-10|
+----------+-------------+------+----------+



In [ ]:
#4. Write the DataFrame as a Delta Table.
customer_df.write.format("delta").mode("overwrite").save("/FileStore/delta/customer_data")

In [ ]:
#5. Convert an existing Parquet file into a Delta Table
df.write.format("parquet").mode("overwrite").save("/FileStore/customer_data.json")

parquet_df = spark.read.format("parquet").load("/FileStore/customer_data.json")
parquet_df.write.format("delta").save("/delta/customer_data_parquet_to_delta")


In [ ]:
#2. Data Management
#1. Load the new_sales_data.csv file into a DataFrame
# Move the file from Workspace to DBFS
dbutils.fs.cp("file:/Workspace/Shared/new_sales_data.csv","dbfs:/FileStore/new_sales_data.csv")

True

In [ ]:
# Load the new_sales_data.csv file from DBFS
new_sales_df = spark.read.format("csv").option("header", "true").load("/FileStore/new_sales_data.csv")
new_sales_df.show()

+-------+----------+----------+--------+--------+-----+
|OrderID| OrderDate|CustomerID| Product|Quantity|Price|
+-------+----------+----------+--------+--------+-----+
|   1009|2024-01-22|      C006|Widget E|      14|20.00|
|   1010|2024-01-23|      C007|Widget F|       6|35.00|
|   1002|2024-01-16|      C002|Widget B|      10|15.75|
+-------+----------+----------+--------+--------+-----+



In [ ]:
#2. Write the new DataFrame as a Delta Table.
new_sales_df.write.format("delta").mode("overwrite").save("/FileStore/delta/new_sales_delta")


In [ ]:
# Load the sales_data.csv into a DataFrame
sales_df = spark.read.format("csv").option("header", "true").load("/FileStore/Sales_data.csv")

# Convert the DataFrame to a Delta Table
sales_df.write.format("delta").save("/delta/sales_data")


In [ ]:
#3. Perform a MERGE INTO operation to update and insert records into the existing Delta table.
from delta.tables import *

# Load the Delta Table
sales_delta = DeltaTable.forPath(spark, "/delta/sales_data")

# Load new sales data
new_sales_df = spark.read.format("csv").option("header", "true").load("/FileStore/new_sales_data.csv")

# Perform the MERGE INTO operation
sales_delta.alias("old").merge(
    new_sales_df.alias("new"),
    "old.OrderID = new.OrderID"
).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()





In [ ]:
# Create a new Delta Table
spark.sql("CREATE TABLE IF NOT EXISTS sales_data_delta USING DELTA LOCATION '/delta/sales_data'")


DataFrame[]

In [ ]:
#diaplay table
spark.sql("""
    SELECT * FROM sales_data_delta
""")

DataFrame[OrderID: string, OrderDate: string, CustomerID: string, Product: string, Quantity: string, Price: string]

In [ ]:
#3. Optimize Delta Table
#1. Apply the OPTIMIZE command on the Delta Table
spark.sql("""
    OPTIMIZE delta.`/delta/sales_data` ZORDER BY (CustomerID)
""")

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,clusteringStats:struct<inputZCubeFiles:struct<numFiles:bigint,size:bigint>,inputOtherFiles:struct<numFiles:bigint,size:bigint>,inputNumZCubes:bigint,mergedFiles:struct<numFiles:bigint,size:bigint>,numOutputZCubes:bigint>,numBins:bigint,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,

In [ ]:
#4. Advanced Features
#1. Use DESCRIBE HISTORY to inspect the history of changes for a Delta Table.
spark.sql("""
    DESCRIBE HISTORY delta.`/delta/sales_data`
""")

DataFrame[version: bigint, timestamp: timestamp, userId: string, userName: string, operation: string, operationParameters: map<string,string>, job: struct<jobId:string,jobName:string,jobRunId:string,runId:string,jobOwnerId:string,triggerType:string>, notebook: struct<notebookId:string>, clusterId: string, readVersion: bigint, isolationLevel: string, isBlindAppend: boolean, operationMetrics: map<string,string>, userMetadata: string, engineInfo: string]

In [ ]:
#2. Use VACUUM to remove old files from the Delta Table.
spark.sql("""
    VACUUM delta.`/delta/sales_data` RETAIN 168 HOURS
""")

DataFrame[path: string]

In [ ]:
#5. Hands-on Exercises
#1. Using Delta Lake for Data Versioning:
#Query historical versions of the Delta Table using Time Travel.
spark.sql("""
    SELECT * FROM delta.`/delta/sales_data` VERSION AS OF 1
""")

DataFrame[OrderID: string, OrderDate: string, CustomerID: string, Product: string, Quantity: string, Price: string]

In [ ]:
#2. Building a Reliable Data Lake with Delta Lake:
#Implement schema enforcement and handle data updates with Delta Lake.
df.write.format("delta").mode("append").option("mergeSchema", "true").save("/delta/sales_data")
